In [1]:
!pip install torch==2.0.1 torchvision==0.15.2

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
import os

import torch
import torch.nn as nn
import torchvision

from glob import glob

from torch.utils.data.dataloader import default_collate
from torchvision import datasets, models, transforms

In [40]:
classes = [os.path.basename(folder) for folder in sorted(glob("../../tmp/test_data/*"))]

classes

['cloudy', 'desert', 'green_area', 'water']

In [4]:
device = torch.device("cpu")

In [5]:
model = models.efficientnet_v2_m()

In [12]:
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, len(classes))

In [13]:
model.load_state_dict(torch.load("../../tmp/efficientnet_v2_m.pth", map_location=device))

<All keys matched successfully>

In [17]:
data_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor()
])

image_dataset = datasets.ImageFolder("../../tmp/test_data", data_transform)

dataloader = torch.utils.data.DataLoader(image_dataset, batch_size=4, shuffle=False, num_workers=2)

In [21]:
inputs, labels = next(iter(dataloader))

outputs = model(inputs)

outputs

tensor([[-0.9691, -0.2161, -5.5451, -4.1060],
        [-0.1002, -5.4098, -1.0635, -0.3583],
        [-5.6530, -6.5186, -3.4168, -1.7497],
        [-2.6396, -2.3899, -2.5033, -1.6456]], grad_fn=<AddmmBackward0>)

In [23]:
_, preds = torch.max(outputs, 1)

preds

tensor([1, 0, 3, 3])

In [43]:
softmax = nn.Softmax(dim=1)

scores = softmax(outputs)

scores = torch.max(scores, 1).values

scores

tensor([0.6683, 0.4632, 0.8214, 0.4407], grad_fn=<MaxBackward0>)

In [42]:
predictions = []
for j in range(inputs.size()[0]):
    predictions.append({
        "correct_class": classes[labels[j]],
        "predicted_class": classes[preds[j]],
        "predicted_score": float(scores[j])
    })

predictions

[{'correct_class': 'cloudy',
  'predicted_class': 'desert',
  'predicted_score': 0.6683429479598999},
 {'correct_class': 'cloudy',
  'predicted_class': 'cloudy',
  'predicted_score': 0.4631504714488983},
 {'correct_class': 'cloudy',
  'predicted_class': 'water',
  'predicted_score': 0.8213943243026733},
 {'correct_class': 'cloudy',
  'predicted_class': 'water',
  'predicted_score': 0.4406794309616089}]

In [46]:
%%time

predictions = []
for inputs, labels in dataloader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    
    _, preds = torch.max(outputs, 1)
    scores = torch.max(softmax(outputs), 1).values

    for j in range(inputs.size()[0]):
        predictions.append({
            "correct_class": classes[labels[j]],
            "predicted_class": classes[preds[j]],
            "predicted_score": float(scores[j])
        })

predictions

CPU times: user 33.2 s, sys: 15.4 s, total: 48.6 s
Wall time: 52.4 s


[{'correct_class': 'cloudy',
  'predicted_class': 'desert',
  'predicted_score': 0.6706036329269409},
 {'correct_class': 'cloudy',
  'predicted_class': 'water',
  'predicted_score': 0.8571392893791199},
 {'correct_class': 'cloudy',
  'predicted_class': 'water',
  'predicted_score': 0.48427340388298035},
 {'correct_class': 'cloudy',
  'predicted_class': 'water',
  'predicted_score': 0.4000720679759979},
 {'correct_class': 'cloudy',
  'predicted_class': 'green_area',
  'predicted_score': 0.4143807590007782},
 {'correct_class': 'cloudy',
  'predicted_class': 'cloudy',
  'predicted_score': 0.8403739929199219},
 {'correct_class': 'cloudy',
  'predicted_class': 'water',
  'predicted_score': 0.6990119218826294},
 {'correct_class': 'cloudy',
  'predicted_class': 'cloudy',
  'predicted_score': 0.40804314613342285},
 {'correct_class': 'cloudy',
  'predicted_class': 'green_area',
  'predicted_score': 0.35332876443862915},
 {'correct_class': 'cloudy',
  'predicted_class': 'water',
  'predicted_sco